<a href="https://colab.research.google.com/github/adwiza/datascience_netology/blob/main/parabola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

# Пример линейная регрессия

Подбираем коэффициенты квадратного уравнения: $y = w1 * x^2 + w2 * x +w3$

# Входные переменные и параметры модели

In [6]:
# tf.placeholder позволяем передавать данные из python в Tensonrflow
# с помощью tf.placeholder мы передаем данные для обучение
x = tf.placeholder(tf.float32, shape=[None], name='x')
y = tf.placeholder(tf.float32, shape=[None], name='y')

In [7]:
# иницивлизируем переменную w
# повторная инициализация приводит к ошибке ??

with tf.vatiable_scope("myscope", reuse=True) as scope:
  